# IMPORTING NECESSARY LIBRARIES

In [1]:
import numpy as np
import keras
import tensorflow
import keras.backend as K
from scipy.misc import imsave , imresize , imread
from scipy.optimize import fmin_l_bfgs_b
from keras.applications import vgg19
from keras.preprocessing.image import load_img , img_to_array
import matplotlib.pyplot as plt
from PIL import Image
import time
import cv2

Using TensorFlow backend.


## IMAGE PREPROCESSING

In [2]:
#An image preprocessing function which will take the the image which is the three dimension into a four dimension where the fourth dimension is the batch size
img_h=512
img_w=512
def preprocess(img_path):
    img = load_img(img_path)
    img = img_to_array(img)
    img = np.resize(img, (img_h, img_w, 3))
    img = img.astype('float64')
    img = np.expand_dims(img, axis=0)
    #process image to fit in the vgg19 network
    img = vgg19.preprocess_input(img)
    return img

In [3]:
CONTENT_IMG_PATH='images/content_image.png'
STYLE_IMG_PATH='images/style_image.png'

In [4]:
#Take the preprocessed image which convert it into array and put the into tensors
content_image=K.variable(preprocess(CONTENT_IMG_PATH))
style_image =K.variable(preprocess(STYLE_IMG_PATH))
#Intializing the generated image
gen_image=K.placeholder(shape=(1,img_h,img_w,3))
#Concatenate tensors along one direction
input_tensor=K.concatenate([content_image,style_image,gen_image] , axis =0)


In [5]:
#Loading model and giving the concatenate input
model=vgg19.VGG19(include_top=False,weights='imagenet',input_tensor=input_tensor)
print("Model loaded")

80142336/80134624 [==============================] - 4s 0us/step
Model loaded


In [6]:
#MAking a dictionary with layer name and its output
output=dict([(layer.name,layer.output) for layer in model.layers])
print('Layers in VGG19 is {}'.format(output))

Layers in VGG19 is {'input_1': <tf.Tensor 'concat:0' shape=(3, 512, 512, 3) dtype=float32>, 'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 512, 512, 64) dtype=float32>, 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 512, 512, 64) dtype=float32>, 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 256, 256, 64) dtype=float32>, 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 256, 256, 128) dtype=float32>, 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 256, 256, 128) dtype=float32>, 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 128, 128, 128) dtype=float32>, 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 128, 128, 256) dtype=float32>, 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 128, 128, 256) dtype=float32>, 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 128, 128, 256) dtype=float32>, 'block3_conv4': <tf.Tensor 'block3_conv4/Relu:0' shape=(3, 128, 128, 256) dtype=float32>, 'block3_p

In [7]:
#Content loss
def content_loss(con,gen) :
    return K.sum(K.square(gen-con))

In [8]:
#Intializing loss as 0
loss=0
#Content loss hyperparameter
alpha=0.025
#Style loss hyperparameter
beta=5.0
#VAriation loss hyperparameter
total_variation_weight = 1.0
#Taking the output from 'block3_conv2' 
layer_features=output['block2_conv2']
#In input tensor we concatenate content image , style image and generated image 
# So [content_image,style_image,generated_image]
content_img_features=layer_features[0 , : , : , :]
generated_img_features=layer_features[2 , : , : , :]
loss += alpha*content_loss(content_img_features,generated_img_features)

In [9]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, gen):
    S = gram_matrix(style)
    G = gram_matrix(gen)
    channels = 3
    size = img_h * img_w
    return K.sum(K.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

In [10]:
feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']

In [11]:
for layer_name in feature_layers:
    layer_features = output[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, combination_features)
    loss += (beta / len(feature_layers)) * sl

In [12]:
# The two losses computed above produce a noisy output.
# Hence, to smoothen the output, we compute the Total Variation Loss
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, 1:, :img_w - 1, :])
    b = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, :img_h - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

# Calculate the total Variation Loss for the Final Output Image (Combination Image)
loss += total_variation_weight * total_variation_loss(gen_image)

In [13]:
grads = K.gradients(loss, gen_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_output = K.function([gen_image], outputs)


def eval_loss_and_grads(x):
    x = x.reshape((1, img_h, img_w, 3))
    outs = f_output([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values


class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_value = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_value = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grads_value = grad_value
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grads_values = np.copy(self.grads_value)
        self.loss_value = None
        self.grads_value = None
        return grads_values

In [14]:
def deprocess_image(img):
    if K.image_data_format() == 'channels_first':
        # For Theano
        img = img.reshape((3, img_h, img_w))
        img = img.transpose((1, 2, 0))
    else:
        img = img.reshape((img_h, img_w, 3))
    # Remove zero-center by mean pixel
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [15]:
evaluator = Evaluator()

# Run L-BFGS optimizer
x = preprocess(CONTENT_IMG_PATH)

iterations = 30

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

    
x=deprocess_image(x)
final_image = Image.fromarray(x)
final_image.save('StyleTransferredImage.png')

Start of iteration 0
Current loss value: 21143724000.0
Iteration 0 completed in 61s
Start of iteration 1
Current loss value: 17298057000.0
Iteration 1 completed in 19s
Start of iteration 2
Current loss value: 16189963000.0
Iteration 2 completed in 18s
Start of iteration 3
Current loss value: 15655305000.0
Iteration 3 completed in 17s
Start of iteration 4
Current loss value: 15381467000.0
Iteration 4 completed in 18s
Start of iteration 5
Current loss value: 15215622000.0
Iteration 5 completed in 19s
Start of iteration 6
Current loss value: 15109140000.0
Iteration 6 completed in 17s
Start of iteration 7
Current loss value: 15025493000.0
Iteration 7 completed in 17s
Start of iteration 8
Current loss value: 14961619000.0
Iteration 8 completed in 17s
Start of iteration 9
Current loss value: 14919695000.0
Iteration 9 completed in 18s
Start of iteration 10
Current loss value: 14885800000.0
Iteration 10 completed in 18s
Start of iteration 11
Current loss value: 14859939000.0
Iteration 11 compl